<a href="https://colab.research.google.com/github/victorthury/tp_final_nlp/blob/main/trabalho_final_victor_thury.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geração de instruções
baseado neste [artigo](https://medium.com/@shahriarsadat71_26111/crafting-your-own-dataset-for-fine-tuning-llama2-in-google-colab-a-step-by-step-guide-part-1-1127002ecf0b)

In [ ]:
!pip install sentence_transformers pypdf faiss-gpu --quiet
!pip install langchain langchain-openai --quiet

%pip install --upgrade --quiet langchain langchain-huggingface text-generation transformers langchainhub sentencepiece jinja2
%pip install --upgrade --quiet bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/85.5 MB ? eta -:--:--ERROR: Operation cancelled by user
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/85.5 MB 67.0 MB/s eta 0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.0 MB

In [ ]:
import pandas as pd
import time
from tqdm import tqdm
import langchain
from langchain_openai import OpenAI
import re
from typing import List, Tuple

import os
os.environ["OPENAI_API_KEY"] = 'retirei-por-motivos-de-segurança'

In [ ]:
from openai import OpenAI
client = OpenAI()

####Para alguns textos foi melhor digitar manualmente

In [ ]:
aluna_gestante = '''A REITORA DA UNIVERSIDADE FEDERAL DO AMAZONAS E PRESIDENTE DO CONSELHO DE ENSINO, PESQUISA E EXTENSÃO, no uso das atribuições estatutárias,
RESOLVE:
Art. 1º APROVAR as normas para o Regime Didático dos cursos de graduação da Universidade Federal do Amazonas - UFAM, contidas nos anexos I e II, partes integrantes indissociáveis desta Resolução.
Art. 2º Esta resolução entra em vigor a partir da sua publicação, revogadas as disposições contidas na Resolução número 021, de 19 de agosto de 1985-CONSEP; Resolução número 006, de 20 de janeiro de 1986 -CONSEP; Portaria GR número 0151, de 07 de janeiro de 2012; Portaria da PROEG número 04/2011, de 25 de abril de 2011.
PLENÁRIO DOS CONSELHOS SUPERIORES DA UFAM "ABRAHAM MOYSÉS COHEN", em Manaus/AM, 03 de maio de 2017.
ANEXO I
NORMAS PARA O REGIME DIDÁTICO DOS CURSOS DE GRADUAÇÃO DA UFAM
CAPÍTULO I DAS DISPOSIÇÕES PRELIMINARES
Art. 1º - Os cursos de graduação oferecidos pela Universidade Federal do Amazonas - UFAM, obedecerão às diretrizes dos respectivos Projetos Pedagógicos, das pré-disposições desta Resolução e da legislação pertinente.
Art. 2º - Os cursos de graduação funcionam em regime de crédito semestral e as horas-aula correspondem às correlações seguintes:
I -1 (um) crédito teórico corresponde a 15 (quinze) horas-aula;
II-1(um) crédito prático corresponde a 30 (trinta) horas-aula.
Parágrafo único A hora-aula dos cursos de graduação, menor sessão de ensino, corresponde a 60 (sessenta) minutos.
Art.3º O período letivo corresponde a 100 (cem) dias de trabalho escolar efetivo, ai considerado a semana de segunda a sábado, excluído o tempo reservado às avaliações finais.
Art.4º Os cursos funcionarão nos turnos prescritos nos respectivos Projetos Pedagógicos, respeitado o máximo de 08 (oito) horas diárias e um intervalo mínimo de 02 (duas) horas entre um turno e outro.
Parágrafo único - Considera-se como um turno o tempo máximo de 6 (seis) horas.
Art.5º A integralização do curso obedecerá ao estabelecido no Projeto Pedagógico respectivo.
CAPÍTULO II DO PLANO DE ENSINO
Art.6º O Plano de Ensino, elaborado de acordo com o Anexo II, deve conter, obrigatoriamente:
I - identificação da disciplina contendo: nome do curso, nome da disciplina, sigla, crédito, pré- requisito, turma, horário, carga horária total (teórica e prática), professor(a), semestre e ano;
II - ementa, conforme o PPC do curso;
III - objetivos: geral e específicos, conforme o PPC do curso;
IV - conteúdo programático/cronograma;
V - metodologia de ensino-aprendizagem;
VI - métodos e instrumentos de avaliação de rendimento;
VII - referências: básica e complementar, conforme o PPC do curso;
VIII - local, data e assinatura do professor;
IX - data da aprovação no Colegiado de Curso e assinatura do Coordenador;
Art.7º As disciplinas oferecidas a cada semestre letivo devem ter os Planos de Ensino aprovados pelo Colegiado de Curso, antes do início das aulas.
§1º O Plano de Ensino de cada disciplina deve ser entregue pelo Professor responsável à Coordenação de Curso, até 10 (dez) dias úteis após a finalização da oferta de disciplinas.
§2º Após a aprovação do Plano de Ensino pelo Colegiado de Curso, o(s) docente(s) responsável (eis) providenciará (ão) a inserção no Diário Eletrônico, obrigatoriamente.
§3º No primeiro dia de aula, o Professor deve expor para o discente o Plano de Ensino, explicando minuciosamente cada item proposto.
CAPÍTULO III DA AVALIAÇÃO
Art. 8º A verificação do rendimento escolar, em cada componente curricular, abrangerá o desempenho do discente nos aspectos de aprendizagem e assiduidade, ambos de caráter eliminatório.
Parágrafo único - Para os fins desta Resolução são consideradas as definições seguintes:
I - aprendizagem: aquisição, pelo discente, dos conhecimentos previstos no Plano de Ensino de cada componente curricular;
II - assiduidade: frequência, pelo discente, de, no mínimo, 75% (setenta e cinco por cento) às aulas e demais atividades previstas no Plano de Ensino de cada componente curricular.
Art. 9º A aferição da aprendizagem nas disciplinas será feita tendo por base os exercícios escolares, e a prova final.
§1º-Os exercícios escolares:
I - serão no mínimo 02 (dois);
II - poderão ter diferentes pesos conforme previamente aprovado no Plano de Ensino;
III - deverão avaliar o todo o conteúdo estabelecido no Plano de Ensino.
§2º A nota final do discente na disciplina será a média ponderada entre a média obtida nos exercícios escolares, com peso 2 (dois), e a nota da prova final, com peso 1 (um).
§3º Será considerado aprovado o discente que obtiver Média Final igual ou superior a 5,0 (cinco vírgula zero).
Art. 10. O discente que obtiver o mínimo de 75% (setenta e cinco por cento) de frequência e Média dos Exercícios Escolares (MEE) igual ou superior a 8,0 (oito vírgula zero) será considerado aprovado na disciplina e dispensado da prova final (PF), resguardado o direito de realizá-la.
Parágrafo único - Para efeito de registro acadêmico, na opção prevista no caput deste artigo, será consignada como Média Final (MF), a Média dos Exercícios Escolares (MEE).
CAPÍTULO IV DAS ATIVIDADES DOCENTES
Art. 11 Na necessidade de ausência do docente este deverá tomar uma das seguintes medidas:
I -reprogramar a atividade em comum acordo com os discentes, fazendo o respectivo registro no Diário Eletrônico; ou
II - fazer substituir-se por outro docente do quadro da UFAM, desde que autorizado pelo Coordenador de Curso ou Chefe de Departamento ou Coordenador Acadêmico.
§1º A ausência de que trata o caput do artigo deverá ser comunicada pelo docente ao superior hierárquico imediato, o qual dará ciência ao Coordenador do Curso e Diretor da Unidade Acadêmica.
§2º O professor ausente somente poderá se fazer substituir, na sala de aula, por docente do quadro ou credenciado nos programas de Pós-graduação da UFAM.
CAPÍTULO V DA ATIVIDADE DISCENTE
SEÇÃO I
DA JUSTIFICATIVA DE AUSÊNCIA
Art. 12. O discente que se ausentar das atividades acadêmicas terá a(s) falta(s) justificada(s) nas hipóteses seguintes:
I - quando for acometido de doença infectocontagiosa, traumática, mórbida ou outras condições de saúde comprovadas por atestado com indicação do Código Internacional de Doenças - CID-, assinado por profissional habilitado na forma da lei;
II - discente em estado de gestação, apresentando desconforto pontual em qualquer fase da gravidez, desde que comprovado por atestado assinado por profissional habilitado na forma da lei;
III - quando em período de luto por morte de ascendente, descendente, colaterais até 2º (segundo) grau, cônjuge ou companheiro, mediante a apresentação do respectivo atestado de óbito e prova de parentesco;
IV - quando, na qualidade de militar da ativa participar de missão militar devidamente comprovada por certidão expedida pela unidade em que estiver prestando serviço,
V - quando, matriculado em órgão de Formação de Reserva, for convocado para exercício ou manobras militares, devidamente comprovada por certidão expedida pela unidade em que estiver prestando serviço;
VI - quando, na qualidade de reservista for convocado para apresentação das reservas ou cerimônia cívica do Dia do Reservista, devidamente comprovada por certidão expedida pela unidade em que estiver prestando serviço;
VII - quando na qualidade de representante na Comissão Nacional de Avaliação da Educação Superior (CONAES), for convocado para participar de reunião, desde que devidamente comprovado;
VIII - quando, for convocado para prestar depoimento judicial ou policial, devidamente comprovado por declaração da autoridade convocante;
IX - quando estiver participando de evento científico, competições desportivas, artística ou acadêmica, desde que comprovada a participação;
X - quando participar voluntariamente de doação de sangue.
Art. 13 O requerimento de justificativa de ausência será obrigatório e dirigido ao(a) Coordenador(a) de Curso, acompanhado do documento disciplinado pelo art. 12, no prazo improrrogável de 5 (cinco) dias úteis, contados da data da atividade acadêmica a ser justificada.
§1º Serão indeferidos os requerimentos que:
I - não estiverem acompanhados dos documentos exigidos no artigo 12;
II - forem intempestivos;
III - não se coadunarem com a justificativa prevista nesta Resolução:
§2º O requerimento de justificativa de ausência poderá:
I - ser assinado por qualquer pessoa maior, desde que represente o discente.
II - encaminhado por via eletrônica, telegráfica ou outro meio idôneo.

Art. 14 - O Coordenador(a) de Curso decidirá sobre o requerimento no prazo de até 05 (cinco) dias úteis, cientificando o professor e o discente ou seu representante da decisão.
Parágrafo único - Do indeferimento do requerimento de justificativa de ausência cabe recurso para o Colegiado do Curso, no prazo improrrogável de 02 (dois) dias após a ciência da decisão.

Art. 15 - O regime de exercícios domiciliares é benefício excepcional deferido aos discentes nas seguintes condições:
I - portadores de afecções que exijam afastamento das atividades acadêmicas superior a 15 dias corridos;
II - gestantes a partir do 8º mês ou em gravidez de risco;
III - parturientes em gozo de licença maternidade.

Art. 16 - Para os fins desta Resolução, terão garantidos o tratamento excepcional os discentes portadores de afecções congênitas ou adquiridas, infecções, traumatismo ou outras condições mórbidas, determinando distúrbios agudos ou agudizados, caracterizado por:
I - incapacidade física relativa, incompatível com a frequência aos trabalhos acadêmicos, desde que se verifique condições necessárias para o prosseguimento da atividade acadêmica;
II - incapacidade psíquica relativa em que se verifique a conservação das condições intelectuais e emocionais necessárias para o prosseguimento da atividade acadêmica;
III - ocorrência isolada ou esporádica, desde que necessárias para o prosseguimento da atividade acadêmica;
IV - intercorrência severa de outros fatores que agravem a situação prevista nos incisos I e II deste artigo, aí compreendidas as síndromes hemorrágicas, asma, cardite, pericardites, afecções osteoarticulares submetidas a correções ortopédicas, nefropatias agudas ou subagudas, afecções reumáticas entre outras.

Art. 17 A estudante em estado gravídico faz jus ao benefício excepcional de exercícios domiciliares, nas condições seguintes:
I - a partir do 8º (oitavo) mês de gestação;
II - durante o período de gozo da licença maternidade;
III - excepcionalmente, em qualquer período gestacional ou pós-parto, desde que seja comprovada a necessidade mediante atestado assinado por profissional habilitado na forma da lei.
Parágrafo único - Qualquer que seja a hipótese autorizadora do benefício previsto no caput deste artigo, o documento a ensejar o deferimento do pedido, será o atestado assinado por profissional habilitado na forma da lei, em que conste o início e o fim do afastamento da discente.

Art. 18 - A solicitação de exercícios domiciliares será requerida ao(a) Coordenador(a) do Curso no prazo de 05 (cinco) dias úteis contados do afastamento indicado pelo profissional atestante habilitado na forma da lei.
Art. 19 - O(a) Coordenador(a) do Curso decidirá sobre o requerimento no prazo de 05 (cinco) dias úteis, cientificando os professores das disciplinas em que o discente esteja matriculado.
Parágrafo único - Do indeferimento do requerimento, cabe recurso para o Colegiado de Curso, no prazo improrrogável de 02 (dois) dias úteis após a ciência da decisão.

Art. 20 - O Coordenador de Curso definirá junto aos professores, conforme as especificidades de cada disciplina, as atividades a serem desenvolvidas pelos discentes em regime de exercícios domiciliares.
§1º O professor da disciplina deverá comunicar para o discente o conteúdo que será cobrado nos exercícios, com pelo menos cinco (05) dias de antecedência.
§2º - Não será deferido regime de exercícios domiciliares para conteúdos que impliquem prática de campo ou estágio obrigatório.
§3º - A viabilidade da aplicação dos exercícios domiciliares, em caso de outros conteúdos práticos, será submetida ao crivo do Coordenador do Curso em conjunto com o docente da respectiva turma.

CAPÍTULO VI DAS DISPOSIÇÕES FINAIS

Art. 21º Este Anexo I entra em vigor quarenta e cinco (45) dias após a publicação da Resolução pertinente, com vistas à adequação dos direitos e atividades aqui previstos e atividades nas Unidades Acadêmicas.
'''

In [ ]:
trancamento_matricula = '''RESOLUÇÃO N° 010/2007

	O REITOR DA UNIVERSIDADE FEDERAL DO AMAZONAS E PRESIDENTE DO CONSELHO ED ADMINISTRAÇÃO, no uso ed suas atribuições estatutárias,

	CONSIDERANDO oque consta no processo n° 008/2007 - CONSAD;

	CONSIDERANDO o Ofício n° 044/2006 - CED, datado de 22/11/2006, encaminhando o processo para apreciação no Conselho de Administração;

	CONSIDERANDO o Ofício n° 053/2007 - CED, datado de 30/05/2007, que encaminhou as alterações no texto do Regimento;

	CONSIDERANDO o Parecer do Relator, aprovado por unanimidade, em reunião ordinária realizada nesta data,


		RESOLVE:
	APROVAR O REGIMENTO INTERNO do CENTRO DE EDUCAÇÃO À DISTÂNCIA/UFAM, na forma aseguir:

	Art.1° - O presente regulamento tem por objetivo disciplinar o funcionamento do Centro de Educação à Distância na Universidade Federal do Amazonas.

	CAPITULO I Dos Objetivos

	Art.2° - São objetivos do Centro de Educação à Distância - CED:
	a. implantar e explorar as interfaces entre a comunicação, as tecnologias da informática, o mundo do trabalho e a educação;
	b. desenvolver, implantar e manter projetos de informatização e recursos computacionais e suporte pedagógico/comunicativo;
	C. produzir material instrucional em diversas mídias utilizando Tecnologias da Inteligência no processo educacional;
	d. criar e manter pesquisas, laboratórios e bibliotecas correspondentes a área das tecnologias aplicadas à educação semipresencial e a distância;
	e. realizar estudos e pesquisas inter-relacionando os saberes formais e da comunidade em projetos de desenvolvimento, de reintegração de recursos do meio ambiente, de cursos de formação continuada e/ou de comunicação entre culturas e etnias diferentes na Amazônia e no país;
	f. promover eventos de socialização de conhecimentos e articulação institucional;
	d. estimular e promover a realização de consultoria técnica, de programas de capacitação e atendimento a solicitação de órgãos e instituições, com ênfase em planejamento de políticas e programas de formação de docentes;
	h. colaborar com outras áreas da Universidade, especialmente em atividades interdisciplinares de EAD;
	i. divulgar e publicar os estudos e pesquisas realizadas pelo CED;
	j. realizar ações que possibilitem o intercâmbio acadêmico, cultural e tecnológico com as demais universidades da Amazônia brasileira e outras instituições de âmbito internacional;
	k. oferecer cursos de formação inicial e continuada, em nível de graduação e pós- graduação na modalidade à distância.

	Art.3° - Para concretizar seus objetivos, o Centro promoverá ações nas linhas de ensino, pesquisa e extensão focados na área de Educação à distância.

	CAPITULO II Da Organização administrativa, tecnológica e Pedagógica

	Art.4° - Na concepção da estrutura do Centro de Educação à Distância da Universidade Federal do Amazonas, serão observados os seguintes fatores:
	I.   características regionais do Estado do Amazonas;
	II.  condições estruturais da Universidade Federal do Amazonas;
	III. sistemática de parceria existentes e possíveis entre as instituições públicas; 	IV. suportes tecnológicos da gestão da aprendizagem me EAD.

	Art.5°-O Centro de Educação à Distância será organizado da seguinte forma:
	a. na sede:
	I.   Estrutura deliberativa;
	II.  Estrutura administrativa;
 	III. Estrutura pedagógica;
	IV.  Estrutura tecnológica;
 	V.   Estrutura física.

	b. nos polos:
	I.   Estrutura administrativa;
	II.  Estrutura pedagógica;
	III. Estrutura tecnológica;
	IV.  Estrutura física;

	Art.6° - São instâncias de gestão em matéria de concepção, execução, acompanhamento e avaliação de projetos:
	I.   Fórum de EAD;
	II.  Coordenação Executiva de Educação à Distância;
	III. Coordenação de Curso;
	IV.  Coordenação de Polo de EAD, que será exercida por professor do quadro efetivo da UFAM e contará com uma estrutura de apoio administrativo.

	Art.7°-OFórum terá a seguinte composição:
	I.    Coordenador Executivo, como presidente;
	II.   Coordenadores dos Cursos;
	III.  Coordenadores dos Polos;
	IV.   Representante da Câmara de Ensino de Graduação;
	V.    Representante da Câmara de Pesquisa e pós-graduação;
	VI.   Representante da Câmara de Extensão.

	Parágrafo Único - As reuniões do Fórum de EAD.
serão realizadas por convocação do Coordenador Executivo ou da maioria simples dos membros.

	Art.8° - O Centro de Educação à Distância terá a seguinte composição:
	I. Diretor Executivo, apoiado por:
	a. Equipe de Apoio Administrativo;
	b. Equipe de planejamento e relações interinstitucionais;
	c. Secretaria Acadêmica;
	d. Coordenador de Polo.

 	II. Coordenação Pedagógica - constituída das Coordenações de Curso - responsável pelas equipes didático pedagógico e de produção de materiais instrucionais, que atuarão por meio dos seguintes núcleos.
	a. Núcleo de pesquisa e produção de material impresso;
	b. Núcleo de supervisão pedagógica, operacional e arte;
	c. Núcleo de avaliação da aprendizagem e correção de provas.

	III. Coordenador Tecnológico, responsável por uma equipe de multimídia - áudio, vídeo e Internet, que atuará por meio dos seguintes núcleos:
	a. Núcleo de produção e apoio à UFAM Virtual;
	b. Núcleo de produção de videoconferência e teleconferência;
	c. Núcleo de produção de vídeo: roteiro, arte gráfica, logística, gravação e edição;
	d. Núcleo de produção de programa para rádio e televisão;
	e. Núcleo de comunicação e marketing do CED/UFAM.

	§ 1° - O Centro poderá propor a participação de um Consultor Externo, especialista em EAD, com atribuições de acompanhar e avaliar ações relacionadas ao Centro de Educação à Distância, conforme plano de trabalho contido em proposta especifica.
	§ 2° - O CED poderá, ainda, ter a participação de estagiários - estudantes de cursos de graduação e pós-graduação da UFAM - em projetos de ensino, pesquisa ou extensão, ou ainda, em atividades da administração do CED.
	§ 3° - A carga horária dos estagiários dedicada ao Programa não deverá ultrapassar 20 (vinte) horas semanais.

	CAPÍTULO III Das Competências

	Art.9° - Compete ao Fórum de EAD:
	I.   elaborar o Regimento Interno do Centro;
	II.  deliberar, em primeira instância, propostas para projetos de curso de ensino, pesquisa e extensão;
	III. articular equipe multidisciplinar para atuação nas diferentes áreas do saber;
	IV.  designar coordenadores dos Polos, que se responsabilizarão pela gestão administrativa e logística das ações;
	V.   deliberar sobre proposta de instalação e manutenção de infraestrutura tecnológica e pedagógica, na sede da Universidade Federal do Amazonas e nos Polos, que deem suporte à teia comunicativa prevista para ocurso;
	VI. deliberar sobre proposta de organização de um sistema comunicativo entre as diferentes instâncias envolvidas: Coordenações de Curso e de Polo, Unidades Acadêmicas da UFAM e Instituições consorciadas.

	Art.10° - São competências do Diretor Executivo de Educação à Distância:
	I.   coordenar o Centro de EAD da UFAM;
	II.   presidir o Fórum de EAD;
	III. coordenar a elaboração de plano anual de ação do CED;
	IV.  acompanhar as ações da Coordenação Pedagógica;
	V. acompanhar as ações da Coordenação Tecnológica;
	VI. implantar Cursos de Formação inicial e continuada, me nível de graduação e pós- graduação na modalidade à distância;
	VII. articular, conceber e apresentar Projetos de Pesquisa que visem a melhoria e a consolidação dos Cursos;
	VIII. implementar Cursos de curta duração, Seminários e Workshops visando ao aperfeiçoamento e à melhoria da qualidade dos Cursos em andamento;
	IX. coordenar os processos de avaliação das ações do Centro;
 	X. elaborar e divulgar relatório semestral das ações realizadas;
	XI. acompanhar e gerenciar os recursos materiais e financeiros oriundos dos projetos e das agências financiadoras, de empresas públicas e privadas, e de prestação de serviços.
	XII. Propor a celebração de convênios e parcerias com órgãos e instituições públicas e/ou privadas que tenham interesses compatíveis com os objetivos dos Cursos.

	Parágrafo Único - No impedimento ou na ausência do Diretor Executivo, assumirá o respectivo cargo o Coordenador Tecnológico.

	Art.11 - São atribuições do Consultor Externo:
	I.   realizar leitura dos documentos produzidos e identificar as equipes e as competências instaladas no ano;
	II.  analisar os artefatos de modelagem e a documentação do desenvolvimento do protótipo da plataforma colaborativa de aprendizagem;
	III. analisar documentos e elaborar pré-texto de avaliação;
	IV.  analisar e discutir a situação atual de desenvolvimento dos produtos na Oficina de Avaliação Técnica e Acadêmica dos Programas do CED/UFAM;
	V.   elaborar relatório técnico de referência para desenvolvimento, utilização de materiais instrucionais e ambientes colaborativos de aprendizagem em ações de formação continuada para professores de educação básica;
	VI.  participar em atividades de avaliação.

	Art.12 - Cabe ao Coordenador de Polo:
	I. encaminhar propostas de projetos de cursos a serem oferecidos nos polos em ensino, pesquisa e extensão;
	II. elaborar relatórios semestrais das ações acadêmico administrativas;
	III. promover articulação da Direção Executiva com as instituições consorciadas no Polo, visando gerenciar as condições pactuadas para o desenvolvimento dos projetos específicos;
	IV. coordenar e manter a infraestrutura tecnológica e pedagógica nos Polos;
participar do Fórum de EAD;
	V. confeccionar relatórios semestrais das ações realizadas;
acompanhar o gerenciamento administrativo dos tutores.

	Art.13 - Cabe ao Coordenador do Curso em consonância com as diretrizes curriculares estabelecidas e em conjunto com professores especialistas e respectivas unidades acadêmicas, elaborar o Projeto Pedagógico do curso com o seguinte formato:

	a. concepção política e pedagógica contendo o contexto sociocultural do projeto; os pressupostos teóricos; a concepção curricular com o detalhamento dos eixos metodológicos do processo ensino aprendizagem; o processo de acesso ao Curso; a estrutura e organização curricular; o sistema de apoio a aprendizagem (orientação acadêmica); o processo de seleção dos orientadores (tutores); o projeto de capacitação me EAD; as condições de trabalho da orientação acadêmica nas áreas especificas e uma proporcionalidade aluno orientador adequada ao projeto pedagógico, acadêmico e administrativo;
	b. detalhamento do sistema de gerenciamento acadêmico (processos de seleção, registro, controle da orientação de aprendizagem, desempenho dos alunos e demais atividades dessa natureza);
	c. avaliação com especificação dos pressupostos conceituais e metodológicos do processo como um todo, incluindo os instrumentos, critérios e estrutura operacional;
	d. especificação do material didático, sua compatibilidade com o projeto pedagógico, a forma de integração das mídias utilizadas, a disponibilidade dos recursos tecnológicos para o grupo social alvo do projeto e autoria intelectual;
	e. indicação para a Direção Executiva da logística de distribuição de material didático;
	f. projeção da estrutura dos custos de produção, constituição dos polos associados e sede, produção e reprodução do material didático, provisão de biblioteca, laboratório, rede, pessoal, etc. especificando as possíveis fontes e recursos previstos;
	g. cronograma físico financeiro de execução do projeto;
	h. indicação do perfil da equipe envolvida: atribuições, titulação, regime de trabalho na instituição e tempo de dedicação ao projeto;
	i. elaboração de relatórios semestrais das ações acadêmico administrativas.

	Parágrafo Único - O projeto deverá estar me consonância ao que determina o MEC, em seus "Indicadores de Qualidade para cursos de Graduação a Distância" e com o que determina a UFAM em suas normas.

	Art.14 - Compete ainda ao Coordenador de Curso:
	I.   propor e implementar projetos de pesquisa e extensão;
	II.  coordenar e acompanhar atividades dos Docentes e Orientadores da Aprendizagem;
	III. elaborar relatórios semestrais das ações acadêmico administrativas;
	IV.  participar da composição do Fórum de EAD.

	CAPÍTULO V Do Quadro de Pessoal
	Art.15 - O quadro de pessoal do Centro será constituído de servidores públicos efetivos da carreira de técnico administrativos me educação e do magistério superior.

	§ 1o - Os servidores que atuam nos polos do Interior serão lotados na respectiva Unidades Acadêmica.
	§2° -Nos polos de Manaus, a lotação dos servidores deverá ser no CED.
	§3° - O pessoal temporário deverá ser remunerado mediante recursos financeiros proveniente de projetos de ensino, pesquisa e extensão.

	Art. 16 - Para o funcionamento das atividades do Programa, poderá, a critério da Coordenação, ser solicitado à Reitoria o remanejamento de pessoal do quadro efetivo da carreira técnica administrativa e docente.

	CAPÍTULO VI
	Das Bolsas de Pesquisa Científica e Extensão

	Art.17 - Os projetos que visem à formação de pesquisadores deverão conter previsão explicita de fontes de financiamento junto às agências financiadoras.

	Art.18 - Tratando-se de projetos de pesquisa e/ou de extensão em parceria com instituições privadas, o CED pleiteará bolsas específicas para seus pesquisadores, professores e alunos.

	CAPÍTULO VII
	Das Disposições Finais e Transitórias

	Art.19 - Pesquisadores ou estudantes poderão fazer uso de equipamentos adquiridos ou cedidos ao CED, desde que autorizados pelo Diretor Executivo e agendados previamente.

	Art.20 - OCED funcionará inicialmente em 02 (duas) salas, nas dependências do 1° andar do Centro de Artes - CAUA.
	Art.21 - Este Regimento estará em vigor na data de sua publicação.
'''

In [ ]:

creditos_optativos = '''
RESOLVE:
Art. l' - Fixar o limite māximo de créditos optativos  que o aluno de g-aduação da Universidade
do Amazonas podera cur  sar em 25s (vinte e cinco por cento) do total de créditos exigi  dos para a
 integralização de seu curso de origem  Parāgrafo Único - Nos cursos em que o limite minimo de  créditos
 optativos é fixado acima dos 25: (vinte e cinco por cer  to) referidos no caput deste artigc,
o limite máximo será o pró  prio limite mínimo fixado para aquele curso.

Art. 20⁃ Esta Resolução entrará em vigor nesta data,  revogadas as đisposições em contrário.
SALA DE REUNIÕES DO CONSELRO DE ENSINO E PESQUISA DA  UNIVERSIDADE DO AMAZONAS, em Manaus,
 7 de dezembro de 1993.
 '''

In [ ]:
retirada_documentos = '''
O PRÓ-REITOR DE ENSINO DE GRADUAÇÃO DA UNIVERSIDADE DO AMAZONAS, uso de suas atribuições normativas

RESOLVE:
Art. 1º - Estabelecer o prazo de 3 (três) meses, a partir da data de conclusão do processo, para o resgate de documentos pessoais em cópia, atestados ou declarações de interesse de seu legítimo proprietário.
§ único - Após esse prazo, os documentos particulares não resgatados deverão ser disponibilizados para reciclagem.
Art. 2º Para readquirir a posse dentro do prazo dos 3 meses acima estipulado, o Requerente deverá provar que é o verdadeiro proprietário do(s) documento(s) solicitado(s), demonstrando essa qualidade através da apresentação, para conferência nesta Pró-Reitoria, de outro documento hábil em original com fotografia.
Art. 3º Ao receber os documentos, o Requerente deverá subscrever um termo de desentranhamento a fim de que fique consignada a devolução ao seu legítimo proprietário.
Art. 4º O requerimento de solicitação de documento deverá ser dirigido à Divisão de Orientação Acadêmica (DOA) no prazo acima estabelecido, devendo conter, além da identificação pessoal, informações exatas e completas, inclusive, se possível, o número do protocolo de ingresso dos documentos nesta Pró-Reitoria, a fim de facilitar a localização.
Dá-se ciência, cumpra-se e publique-se.
PRÓ-REITORIA DE ENSINO DE GRADUAÇÃO DA UNIVERSIDADE DO AMAZONAS, em 3 maio de 2002.
'''

In [ ]:
matricula = '''
RESOLVE:

Artigo 1º - PROIBIR a inclusão em discilinasturmas de alunos da graduação que não
estejam oficiaimerte matriculados, tornando-se nulo qualquer inclusão, trabalho acadêmico ou
xercicio escolar que tenha so realizado em desrespelo aos procegimentos é prazos formais
de matricu
8 1º - Excetuam.se do disposto no caput deste arigo alunos que, comprovadamente,
estejam com recurso interposto, em razão de erro Instucional permitindo-se para estes
apenas a fequência enquanto não jugado o recurso

82º - Caso seja contimada a matricula na disciinanuma, deverá ser garantida ao
aluno a realização, excepcional, de segunda chamada dos exercícios escolares porventura
reglizados antes da datada eletvação d matricula

Artigo 2º - Esta Resolução entra em vigor nesta data

Plenário Moysés Abraham CoheryUFAM, em Manaus, 1º de Agosto de 2007.

fa Uns

Bruco Osbomo
Presidente

TESS o CE EINSEPE DE o
'''

## aplicando sanitização

In [ ]:
df = pd.read_pickle('data_sanitized.pkl')
df['sanitized_text'] = df['sanitized_text'].str.replace('\n', '')
df['assunto'] = [text.replace('_', ' ') for text in df['assunto']]
df['sanitized_text'][5] = ' '.join(aluna_gestante.lower().replace('\n', ' ').split())
df['sanitized_text'][23] = ' '.join(trancamento_matricula.lower().replace('\n', ' ').split())
df['sanitized_text'][24] = ' '.join(creditos_optativos.lower().replace('\n', ' ').split())
df['sanitized_text'][38] = ' '.join(retirada_documentos.lower().replace('\n', ' ').split())
df['sanitized_text'][40] = ' '.join(aluna_gestante.lower().replace('\n', ' ').split())
df['sanitized_text'][41] = ' '.join(aluna_gestante.lower().replace('\n', ' ').split())
df['sanitized_text'][57] = ' '.join(matricula.lower().replace('\n', ' ').split())

In [ ]:
documents = df.to_dict('records')

In [ ]:
def extract_instruction_response_pairs(string: str):#-> Tuple[List[str], List[str]]:
    """
    Extracts pairs of instructions and responses from a JSON-formatted string.

    Parameters:
        - json_string (str): A string containing JSON-formatted instruction and response pairs.

    Returns:
        - instructions (list): A list of extracted instructions.
        - responses (list): A list of extracted responses corresponding to the instructions.
    """

    pattern = r'{"Instruction": "(.*?)", "Response": "(.*?)"}'

    # Use re.findall to extract matches
    matches = re.findall(pattern, string)

    # Extract lists of "Instruction" and "Response"
    instructions = [match[0] for match in matches]
    responses = [match[1] for match in matches]

    return instructions, responses

In [ ]:
documents[0].keys()

dict_keys(['assunto', 'text', 'sanitized_text', 'len'])

In [ ]:
1000 / len(documents)

16.39344262295082

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": f"Você irá formatar dados do regulamento interno da ufam de assunto {documents[0]['assunto']}, a saída deve ser um array de 5 json, cada json teve ter uma pergunta e uma resposta"},
    {"role": "user", "content": "Art 2º O estágio poderá ser obrigatória ou não-obrigatório, conforma determinação das diretrizes curiculares. a etapa, modalidade e área de ensino e do projeto pedagógico do curso,"},
    {"role": "assistant", "content": "{'Instruction': 'Quais as modalidades de estágio', 'Response': 'Pode ser obrigatório e não obrigatório'}"},
    {"role": "user", "content": documents[0]['sanitized_text']}
  ]
)

In [ ]:
json.loads(response.choices[0].message.content)

[{'pergunta': 'O que é o Núcleo Docente Estruturante (NDE) de um curso de graduação na UFAM?',
  'resposta': 'O Núcleo Docente Estruturante (NDE) de um curso de graduação é uma instância consultiva, constituída por professores do próprio curso, com atribuições acadêmicas de acompanhamento, consolidação e contínua atualização do projeto pedagógico.'},
 {'pergunta': 'Quais são as principais atribuições do Núcleo Docente Estruturante (NDE) na UFAM?',
  'resposta': 'As principais atribuições do NDE são zelar pela observância das diretrizes curriculares nacionais, contribuir para a avaliação e reestruturação do projeto pedagógico, propor incentivos ao desenvolvimento da pesquisa e extensão, e acompanhar a implantação e desenvolvimento das necessidades da graduação e da área de conhecimento do curso.'},
 {'pergunta': 'Como é composto o Núcleo Docente Estruturante (NDE) na UFAM?',
  'resposta': 'O NDE é composto pelo coordenador do curso e por no mínimo cinco professores efetivos do curso, co

In [ ]:
All_instructions = []
All_reponses = []

for i, document in enumerate(documents):
  for idx in tqdm(range(3)):
    response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "system", "content": f"Você irá formatar dados do regulamento interno da ufam de assunto {document['assunto']}, foque nos artigos para gerar as perguntas. A saída deve ser um array de 6 json usando aspas duplas, cada json teve ter um campo pergutna e um campo resposta. Tente diversificar um pouco as perguntas. Lembre-se, deve devolver um array, o json usa aspas duplas e não precisa formatar para markdown."},
        {"role": "user", "content": "Art 2º O estágio poderá ser obrigatória ou não-obrigatório, conforma determinação das diretrizes curiculares. a etapa, modalidade e área de ensino e do projeto pedagógico do curso,"},
        {"role": "assistant", "content": "{'Instruction': 'Quais as modalidades de estágio', 'Response': 'Pode ser obrigatório e não obrigatório'}"},
        {"role": "user", "content": document['sanitized_text']}
      ]
    )
    objs = json.loads(response.choices[0].message.content)
    for obj in objs:
      All_instructions.append(obj['pergunta'])
      All_reponses.append(obj['resposta'])

100%|██████████| 3/3 [00:32<00:00, 10.81s/it]


In [ ]:
df = pd.DataFrame()
df["Intructions"] = All_instructions
df["Responses"] = All_reponses

In [ ]:
df.to_pickle('instruções.pkl')

para evitar rodar tudo isso, vou apenas ler do pkl

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_pickle('instruções.pkl')

In [ ]:
df

,Intructions,Responses
0,O que regulamenta a resolução nº 062/2011 da U...,A resolução nº 062/2011 trata da criação e reg...
1,Quais são as atribuições do Núcleo Docente Est...,As atribuições do Núcleo Docente Estruturante ...
2,"Segundo o Art. 4º, como deve ser composto o Nú...",O Núcleo Docente Estruturante deve ser compost...
3,O que estabelece o Art. 5º sobre a composição ...,O Art. 5º estabelece que o Núcleo Docente Estr...
4,Com que frequência o Núcleo Docente Estruturan...,O Núcleo Docente Estruturante deve se reunir u...
...,...,...
1093,Quais são os objetivos principais do PET?,Desenvolver atividades acadêmicas de excelênci...
1094,Como são formados os grupos PET?,Os grupos PET são formados a partir de cursos ...
1095,Quem coordena a implementação e execução do PET?,A implementação e execução do PET são coordena...
1096,Quais são os requisitos para ser um tutor de g...,O tutor deve pertencer ao quadro permanente da...


# Fine tuning

baseado neste [artigo](https://medium.com/@shahriarsadat71_26111/crafting-your-own-dataset-for-fine-tuning-llama2-in-google-colab-a-step-by-step-guide-part-2-8059a063b5e0)

Caso vá executar o modelo, pule para a seção em que carrego a partir do huggingface

In [ ]:
!pip install -U transformers datasets trl peft accelerate bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requir

In [ ]:
## important imports
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
import datasets
import pandas as pd
import numpy as np
import time

In [ ]:
df = pd.read_pickle('instruções.pkl')

In [ ]:
df["formatted_instruction"] = df.apply(lambda x: f"### Instruction:\n{x['Intructions']}\n\n### Response:\n{x['Responses']}", axis=1)


In [ ]:
df.head()

,Intructions,Responses,formatted_instruction
0,O que regulamenta a resolução nº 062/2011 da U...,A resolução nº 062/2011 trata da criação e reg...,### Instruction:\nO que regulamenta a resoluçã...
1,Quais são as atribuições do Núcleo Docente Est...,As atribuições do Núcleo Docente Estruturante ...,### Instruction:\nQuais são as atribuições do ...
2,"Segundo o Art. 4º, como deve ser composto o Nú...",O Núcleo Docente Estruturante deve ser compost...,"### Instruction:\nSegundo o Art. 4º, como deve..."
3,O que estabelece o Art. 5º sobre a composição ...,O Art. 5º estabelece que o Núcleo Docente Estr...,### Instruction:\nO que estabelece o Art. 5º s...
4,Com que frequência o Núcleo Docente Estruturan...,O Núcleo Docente Estruturante deve se reunir u...,### Instruction:\nCom que frequência o Núcleo ...


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [ ]:
base_model = "NousResearch/Llama-2-7b-chat-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
dataset = datasets.Dataset.from_pandas(df)

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# a celula abaixo demora a rodar
start = time.time()
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1
print("time for model load: {} seconds".format(time.time()-start))

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

time for model load: 175.87348127365112 seconds


In [ ]:
def calculate_model_size(model):
    total_size = 0
    for param in model.parameters():
        param_size = param.numel() * param.element_size()  # size in bytes
        total_size += param_size

    total_size_in_mb = total_size / (1024**2)  # convert to megabytes
    return total_size_in_mb

# Example usage:
model_size_mb = calculate_model_size(model)
print(f"Model size: {model_size_mb:.2f} MB")

Model size: 3588.51 MB


In [ ]:
peft_params = LoraConfig(
    lora_alpha=16, ##The alpha parameter for Lora scaling.
    lora_dropout=0.1, ##The dropout probability for Lora layers.
    r=64, ## Most important hyperparameter. Lora attention dimension, or "rank"
    bias="none",
    task_type="CAUSAL_LM" ## next token prediction: the autoregressive approach.
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="formatted_instruction",
    max_seq_length=128,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1098 [00:00<?, ? examples/s]

In [ ]:
start = time.time()
output = trainer.train()
print("Time taken: ", time.time()-start)

Step,Training Loss
25,2.199900
50,1.633500
75,1.564600
100,1.349500
125,1.511600
150,1.343800
175,1.361800
200,1.294200
225,1.388800
250,1.186200


Time taken:  1309.1698298454285


In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,
                max_length=200)
prompt = "###Instruction:\nO que diz sobre trote na ufam? \n\n###Response\n:"
gen_text = pipe(prompt)
print(gen_text[0]['generated_text'][len(prompt):])

O trote não é uma atividade obrigatória.

###Instruction:
Qual é o prazo para a realização do trote?

###Response:
O trote pode ser realizado em qualquer período do ano.

###Instruction:
Qual é o limite de trote?

###Response:
Não há limite de trote.

###Instruction:
Qual é o tipo de trote?

###Response:
O trote pode ser trote obrigatório ou não-obrigatório.

###Instruction:
Qual é o objetivo do trote?

###Response:
O trote tem como objetivo promover a integração social e profissional entre os


# Salvando o modelo

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from huggingface_hub import HfApi
from huggingface_hub.utils import HfHubHTTPError
api = HfApi()
model_repo_name = "victorthury/tp_final"

In [ ]:
import urllib.error

In [ ]:
try:
  api.create_repo(repo_id=model_repo_name)
except HfHubHTTPError as err:
  print(err)

409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-66ae69b3-3303beec4f99439f270e280f;36c0343b-0e67-4188-8047-e19669a1ad1b)

You already created this model repo


In [ ]:
trainer.model.save_pretrained('trainer/tp_final')

In [ ]:
api.upload_folder(
    folder_path='/content/trainer',
    repo_id=model_repo_name
)

adapter_model.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/victorthury/tp_final/commit/e3d044963a21a707232cbf8c16e76fa90acf7574', commit_message='Upload folder using huggingface_hub', commit_description='', oid='e3d044963a21a707232cbf8c16e76fa90acf7574', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_carregado = AutoModelForCausalLM.from_pretrained(
    '/content/trainer/tp_final',
    quantization_config=quant_config,
    device_map={"": 0}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline(task="text-generation", model=model_carregado, tokenizer=tokenizer,
                max_length=200)
prompt = "###Instruction:\nO que diz sobre trote na ufam? \n\n###Response\n:"
gen_text = pipe(prompt)
print(gen_text[0]['generated_text'][len(prompt):])

O trote não é considerado uma condição de exceção.

###Instruction:
Qual é a condição para que o trote seja considerado?

###Response:
O trote deve ser homologado pelo chefe de departamento.

###Instruction:
Qual é o prazo para o chefe de departamento homologar o trote?

###Response:
O chefe de departamento tem 3 dias para homologar o trote.

###Instruction:
Qual é o que acontece se o chefe de departamento não homologar o trote?

###Response:
O trote não será homologado.

###Instruction:
Qual é a condição para que o trote


In [ ]:
api.upload_folder(
    folder_path='tp2',
    repo_id=model_repo_name
)

# Carregando modelo com fine tuning

In [ ]:
!git clone https://huggingface.co/victorthury/tp_final/

Cloning into 'tp_final'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 2), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (22/22), 9.58 KiB | 228.00 KiB/s, done.


In [ ]:
!pip install -U transformers datasets trl peft accelerate bitsandbytes --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
## important imports
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
import datasets
import pandas as pd
import numpy as np
import time

In [ ]:
compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
model_carregado = AutoModelForCausalLM.from_pretrained(
    '/content/tp_final/tp_final',
    quantization_config=quant_config,
    device_map={"": 0}
)

pipe = pipeline(task="text-generation", model=model_carregado, tokenizer=tokenizer,
                max_length=200)
prompt = "###Instruction:\nO trote é permitido na ufam? \n\n###Response\n:"
gen_text = pipe(prompt)
print(gen_text[0]['generated_text'][len(prompt):])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

NameError: name 'tokenizer' is not defined